### Loading dataset

In [2]:
import pandas as pd

In [3]:
path = '/content/drive/MyDrive/praksa_manu/distilroberta_model_training/tokenized_dataset.csv'

data = pd.read_csv(path)

data.head()

,Tweet,Close,Month,Year,Month_sin,Month_cos,Year_sin,Year_cos,Tokenized_Tweet,Input_IDs
0,Mainstream media has done an amazing job at br...,0.377748,0.727273,1.0,-1.0,-1.836970e-16,0.955237,0.295841,"['Main', 'stream', 'Ġmedia', 'Ġhas', 'Ġdone', ...","[0, 31359, 8656, 433, 34, 626, 41, 2770, 633, ..."
1,Tesla delivery estimates are at around 364k fr...,0.377748,0.727273,1.0,-1.0,-1.836970e-16,0.955237,0.295841,"['Tesla', 'Ġdelivery', 'Ġestimates', 'Ġare', '...","[0, 39971, 2996, 2785, 32, 23, 198, 38930, 330..."
2,3/ Even if I include 63.0M unvested RSUs as of...,0.377748,0.727273,1.0,-1.0,-1.836970e-16,0.955237,0.295841,"['3', '/', 'ĠEven', 'Ġif', 'ĠI', 'Ġinclude', '...","[0, 246, 73, 1648, 114, 38, 680, 5549, 4, 288,..."
3,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,0.377748,0.727273,1.0,-1.0,-1.836970e-16,0.955237,0.295841,"['@', 'Real', 'Dan', 'OD', 'ow', 'd', 'Ġ@', 'W...","[0, 1039, 17105, 25887, 7111, 1722, 417, 787, ..."
4,"@RealDanODowd @Tesla Stop trying to kill kids,...",0.377748,0.727273,1.0,-1.0,-1.836970e-16,0.955237,0.295841,"['@', 'Real', 'Dan', 'OD', 'ow', 'd', 'Ġ@', 'T...","[0, 1039, 17105, 25887, 7111, 1722, 417, 787, ..."


In [4]:
data = data.drop(columns=['Tokenized_Tweet', 'Input_IDs'])

In [5]:
data.head()

,Tweet,Close,Month,Year,Month_sin,Month_cos,Year_sin,Year_cos
0,Mainstream media has done an amazing job at br...,0.377748,0.727273,1.0,-1.0,-1.836970e-16,0.955237,0.295841
1,Tesla delivery estimates are at around 364k fr...,0.377748,0.727273,1.0,-1.0,-1.836970e-16,0.955237,0.295841
2,3/ Even if I include 63.0M unvested RSUs as of...,0.377748,0.727273,1.0,-1.0,-1.836970e-16,0.955237,0.295841
3,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,0.377748,0.727273,1.0,-1.0,-1.836970e-16,0.955237,0.295841
4,"@RealDanODowd @Tesla Stop trying to kill kids,...",0.377748,0.727273,1.0,-1.0,-1.836970e-16,0.955237,0.295841


### Preparing the tokenizer and model


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [7]:
tokenizer_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [9]:
def tokenize_and_create_attention_mask(tweet, max_length=128):

    encoding = tokenizer(tweet, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')

    token_ids = encoding['input_ids'].squeeze().tolist()

    attention_mask = encoding['attention_mask'].squeeze().tolist()

    return token_ids, attention_mask

In [10]:
data[['Token_IDs', 'Attention_mask']] = data['Tweet'].apply(lambda x: pd.Series(tokenize_and_create_attention_mask(x)))

print(data[['Tweet', 'Token_IDs', 'Attention_mask']].head())

                                               Tweet  \
0  Mainstream media has done an amazing job at br...   
1  Tesla delivery estimates are at around 364k fr...   
2  3/ Even if I include 63.0M unvested RSUs as of...   
3  @RealDanODowd @WholeMarsBlog @Tesla Hahaha why...   
4  @RealDanODowd @Tesla Stop trying to kill kids,...   

                                           Token_IDs  \
0  [0, 31359, 8656, 433, 34, 626, 41, 2770, 633, ...   
1  [0, 39971, 2996, 2785, 32, 23, 198, 38930, 330...   
2  [0, 246, 73, 1648, 114, 38, 680, 5549, 4, 288,...   
3  [0, 1039, 17105, 25887, 7111, 1722, 417, 787, ...   
4  [0, 1039, 17105, 25887, 7111, 1722, 417, 787, ...   

                                      Attention_mask  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..

In [19]:
input_ids = torch.tensor(data['Token_IDs'].tolist())

In [20]:
attention_mask = torch.tensor(data['Attention_mask'].tolist())

In [21]:
inputs = {
    "input_ids": input_ids,
    "attention_mask": attention_mask
}

In [ ]:
outputs = model(**inputs)

predictions = torch.argmax(outputs.logits, axis=-1)

predictions_list = predictions.tolist()

data['Predictions'] = predictions_list

path = '/content/drive/MyDrive/praksa_manu/distilroberta_model_training/dataset_with_predictions.csv'

data.to_csv(path, index=False)

It appears a problem here. The dataset is too big and when i try to predict the data my RAM memory is filling up and the session is aborting. As a solution to this I will try to read the dataset partially and predict the data like that.

### Predicting the data partially


In [11]:
import numpy as np

In [12]:
chunk_size = 512

data_chunks = np.array_split(data, len(data) // chunk_size + 1)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [13]:
chunk_1 = data_chunks[0]
print(chunk_1.head())

                                               Tweet     Close     Month  \
0  Mainstream media has done an amazing job at br...  0.377748  0.727273   
1  Tesla delivery estimates are at around 364k fr...  0.377748  0.727273   
2  3/ Even if I include 63.0M unvested RSUs as of...  0.377748  0.727273   
3  @RealDanODowd @WholeMarsBlog @Tesla Hahaha why...  0.377748  0.727273   
4  @RealDanODowd @Tesla Stop trying to kill kids,...  0.377748  0.727273   

   Year  Month_sin     Month_cos  Year_sin  Year_cos  \
0   1.0       -1.0 -1.836970e-16  0.955237  0.295841   
1   1.0       -1.0 -1.836970e-16  0.955237  0.295841   
2   1.0       -1.0 -1.836970e-16  0.955237  0.295841   
3   1.0       -1.0 -1.836970e-16  0.955237  0.295841   
4   1.0       -1.0 -1.836970e-16  0.955237  0.295841   

                                           Token_IDs  \
0  [0, 31359, 8656, 433, 34, 626, 41, 2770, 633, ...   
1  [0, 39971, 2996, 2785, 32, 23, 198, 38930, 330...   
2  [0, 246, 73, 1648, 114, 38, 680, 55

In [ ]:
all_predictions = []

for chunk in data_chunks:

    input_ids = torch.tensor(chunk['Token_IDs'].tolist())
    attention_mask = torch.tensor(chunk['Attention_mask'].tolist())


    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(outputs.logits, axis=-1)


    all_predictions.extend(predictions.tolist())

data['Predictions'] = all_predictions
data.to_csv('predictions_output.csv', index=False)